- [Open with Google Colab](https://colab.research.google.com/github/zaq9/case/blob/master/case_2018dec_put.ipynb



In [0]:
#Google colabo用パッケージのインストール

!pip install -q pulp

In [0]:
import pulp as pu

op_names = ["P21000", "P20500", "P20000", "P19500", "P19000",
            "P18500", "P18000", "P17000", "P16000", "P15000", "P14000"
            ]
#2018年１２月PUTの市場データ＋badCase（日経17000、IV80)のデータ

data = {
    "dec28": [1007, 652, 415, 255, 155, 97, 62, 27, 10, 5, 1],
    "dec26": [1922, 1580, 1170, 850, 605, 415, 290, 145, 70, 34, 21],
    "dec21": [880, 615, 395, 245, 155, 91, 53, 22, 8, 4, 1],
    "dec18": [255, 140, 83, 50, 29, 20, 13, 8, 3, 2, 1],
    "dec04": [93, 62, 44, 30, 23, 20, 16, 12, 8, 6, 4],
    "dec03": [170, 115, 76, 54, 40, 30, 24, 16, 10, 8, 5],
    "bad_case": [3125, 2684, 2263, 1869, 1508, 1183, 899, 464, 198, 67, 17],
}

ops = [pu.LpVariable(s, 0, 200, pu.LpInteger) for s in op_names]
ops_n = [pu.LpVariable(s+'_n', 0, 20, pu.LpInteger) for s in op_names]

def v(s): 
    return pu.lpSum(pu.lpDot(ops, data[s])) - pu.lpSum(pu.lpDot(ops_n, data[s]))


In [5]:
m = pu.LpProblem('Max', sense=pu.LpMaximize)
cost = v('dec18')
m += cost <= 100
m += pu.lpSum(ops_n) <= 20  # 制約：合計売り枚数　２０枚以下

for s in data.keys(): m += v(s) >= -100

m += v('dec26') - cost

m.solve()
print(f" Max = {m.objective.value()}")

for op in ops:
    if op.value() != 0: print(f"{op.name}[{op.value() :.0f}]")
for op in ops_n:
    if op.value() != 0: print(f"{op.name[0:6]}[{-op.value() :.0f}]")


 Max = 75208.0
P19000[63]
P18000[200]
P16000[200]
P14000[173]
P21000[-20]
